In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime
import json

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [2]:
from FormatEvents_Functions import *

In [3]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [4]:
durations = ['0.5', '1', '2', '3', '6', '12', '24']

### Create one dataframe containing both present and future

In [5]:
df_long = pd.read_csv(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics.csv")
nimrod_long = pd.read_csv(home_dir +  f"ProcessedData/AMAX_Events/NIMROD_30mins/all_events_characteristics.csv")

In [6]:
nimrod_long['max_precip'] = 1
nimrod_long['mean_precip'] = 1
nimrod_long['Climate'] = 'NIMROD'

In [7]:
present = df_long[df_long['Climate'] =='Present']
nimrod_long_with_present = pd.concat([present, nimrod_long])

### Remove entries which are less than 1.5 hours

In [17]:
# df_long_with_short_durations_kept = df_long.copy()

In [8]:
df_long = df_long[df_long['duration'] >=1.5]
nimrod_long = nimrod_long[nimrod_long['duration'] >=1.5]
# present = present[present['duration'] >=1.5]
# future = future[future['duration'] >=1.5]
# # nan_rows = df_long[df_long['D50'].isna()]

# Create grouped results, for all events (no duplicates for durations)
### Group by gauge, climate

### NIMROD vs UKCP18 Present

In [9]:
group_by_columns = ['Climate', 'gauge_num']
grouped_by_gauge_allevents = group_data_calc_means(nimrod_long_with_present, 'D50_new', group_by_columns)
grouped_by_gauge_allevents_changes = find_change_values_in_groups_nimrod(grouped_by_gauge_allevents, group_by_columns, 'All')
grouped_by_gauge_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/grouped_by_gauge_changes_allevents.csv", index=False)

### UKCP18 Present vs Future

In [10]:
group_by_columns = ['Climate', 'gauge_num']
grouped_by_gauge_allevents = group_data_calc_means(df_long, 'D50_new', group_by_columns)
grouped_by_gauge_allevents_changes = find_change_values_in_groups_new(grouped_by_gauge_allevents, group_by_columns, 'All')
grouped_by_gauge_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_changes_allevents.csv", index=False)

## Group by season, gauge, climate

In [11]:
group_by_columns = ['Climate', 'gauge_num', 'season']
grouped_by_gauge_season_allevents = group_data_calc_means(nimrod_long_with_present, 'D50_new', group_by_columns)
grouped_by_gauge_season_allevents_changes = find_change_values_in_groups_nimrod(grouped_by_gauge_season_allevents, group_by_columns, 'All')
grouped_by_gauge_season_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/grouped_by_gauge_season_changes_allevents.csv", index=False)

In [12]:
group_by_columns = ['Climate', 'gauge_num', 'season']
grouped_by_gauge_season_allevents = group_data_calc_means(df_long, 'D50_new', group_by_columns)
grouped_by_gauge_season_allevents_changes = find_change_values_in_groups_new(grouped_by_gauge_season_allevents, group_by_columns, 'All')
grouped_by_gauge_season_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_season_changes_allevents.csv", index=False)

# Create grouped results, for all events (for each duration separately)
## Group by gauge, climate and by season, gauge, climate
### NIMROD vs UKCP18 Present

In [30]:
each_dur_per_climate_changes = []
each_dur_per_climate_and_season_changes = []

# For each duration in turn
for duration in ['16hr+', '7-16hr', '<=7hr']:
    # Get data for just this duration
    this_dur = nimrod_long_with_present[nimrod_long_with_present["duration_category_onemore"] == duration]
    # this_dur = this_dur[this_dur['duration']>8]
    # Summary of events at each gauge, for this duration, one for present, one for future
    summary_per_climate = group_data_calc_means(this_dur, 'D50_new', ['Climate', 'gauge_num'])
    # Summary of events at each gauge, for this duration, one for each season for present, one for each season for future
    summary_per_climate_and_season = group_data_calc_means(this_dur, 'D50_new', ['Climate', 'gauge_num', 'season'])
    # Reformat, so one row per gauge, with change between present and future in the columns
    summary_per_climate_changes = find_change_values_in_groups_nimrod(summary_per_climate, ['Climate', 'gauge_num'], duration)
    # Reformat, so four rows (each season) per gauge, with change between present and future in the columns
    summary_per_climate_season_changes = find_change_values_in_groups_nimrod(summary_per_climate_and_season, ['Climate', 'gauge_num', 'season'], duration)
    
    ## Add to lists
    each_dur_per_climate_changes.append(summary_per_climate_changes)
    each_dur_per_climate_and_season_changes.append(summary_per_climate_season_changes)

In [34]:
total_cats = pd.concat(each_dur_per_climate_changes)
total_season_cats = pd.concat(each_dur_per_climate_and_season_changes)

In [36]:
total_cats.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/grouped_by_gauge_changes_bydurcategory.csv", index=False)
total_season_cats.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/grouped_by_gauge_season_changes_bydurcategory.csv", index=False)

### UKCP18 Present vs Future

In [13]:
each_dur_per_climate_changes = []
each_dur_per_climate_and_season_changes = []

# For each duration in turn
for duration in ['16hr+', '7-16hr', '<=7hr']:
    # Get data for just this duration
    this_dur = df_long[df_long["duration_category_onemore"] == duration]
    print(this_dur['duration'].mean())
    # this_dur = this_dur[this_dur['duration']>8]
    # Summary of events at each gauge, for this duration, one for present, one for future
    summary_per_climate = group_data_calc_means(this_dur, 'D50_new', ['Climate', 'gauge_num'])
    # Summary of events at each gauge, for this duration, one for each season for present, one for each season for future
    summary_per_climate_and_season = group_data_calc_means(this_dur, 'D50_new', ['Climate', 'gauge_num', 'season'])
    # Reformat, so one row per gauge, with change between present and future in the columns
    summary_per_climate_changes = find_change_values_in_groups_new(summary_per_climate, ['Climate', 'gauge_num'], duration)
    # Reformat, so four rows (each season) per gauge, with change between present and future in the columns
    summary_per_climate_season_changes = find_change_values_in_groups_new(summary_per_climate_and_season, ['Climate', 'gauge_num', 'season'], duration)
    
    ## Add to lists
    each_dur_per_climate_changes.append(summary_per_climate_changes)
    each_dur_per_climate_and_season_changes.append(summary_per_climate_season_changes)

26.041057772133247
11.469964304323144
3.9885129058476694


In [14]:
total_cats = pd.concat(each_dur_per_climate_changes)
total_season_cats = pd.concat(each_dur_per_climate_and_season_changes)

In [15]:
total_cats.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/grouped_by_gauge_changes_bydurcategory.csv", index=False)
total_season_cats.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/grouped_by_gauge_season_changes_bydurcategory.csv", index=False)

### Save original data
Don't do this higher up, because the json.dumps thing messes up the formatting for later stages of the code

In [29]:
# Create a copy and convert lists to JSON strings before saving - not doing this, messed up formatting a bit
# df_long['dur_for_which_this_is_amax'] = df_long['dur_for_which_this_is_amax'].apply(json.dumps)
# df_long_with_short_durations_kept['dur_for_which_this_is_amax'] = df_long_with_short_durations_kept['dur_for_which_this_is_amax'].apply(json.dumps)
# df_long_with_short_durations_kept.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics_shortdurationskept_new.csv", index=False)

In [16]:
df_long.to_csv(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics.csv", index=False)
nimrod_long.to_csv(home_dir +  f"ProcessedData/AMAX_Events/NIMROD_30mins/all_events_characteristics.csv", index=False)